In [1]:
from math import pi, exp, sqrt, erf
import numpy as np
def N(x):
    return (2*x/pi)**0.75
def F0(t):
    return 1.0 if t<1e-10 else 0.5 *sqrt(pi/t)*erf(sqrt(t))
def u(x,y):
    return x*y/(x + y)
def S(a, b, A, B):
    A = np.array(A, dtype = float)
    B = np.array(B, dtype = float)
    Rab2 = np.dot(A - B, A - B)
    return N(a)*N(b)*(pi/(a+b))**1.5 * exp(-u(a,b)*Rab2)
def T(a,b,A,B):
    A = np.array(A, dtype = float)
    B = np.array(B, dtype = float)
    Rab2 = np.dot(A - B, A - B)
    return N(a)*N(b)*u(a,b)*(3-2*u(a,b)*Rab2)*(pi/(a+b))**1.5*exp(-u(a,b)*Rab2)
def V(a,b,A,B,C,Z):
    A = np.array(A, dtype = float)
    B = np.array(B, dtype = float)
    C = np.array(C, dtype = float)
    Rab2 = np.dot(A - B, A - B)
    P = (a*A+b*B)/(a+b)
    Rpc2 = np.dot(P-C, P-C)
    return -N(a) * N(b) * (2*pi*Z/(a+b)) * exp(-u(a,b) * Rab2) * F0((a+b) * Rpc2)
def two_electron_ERI (a,b,c,d,A,B,C,D):
    A = np.array(A, dtype = float)
    B = np.array(B, dtype = float)
    C = np.array(C, dtype = float)
    D = np.array(D, dtype = float)
    p1 = a+b
    p2 = c+d
    P = (a*A+b*B)/(p1)
    Q = (c*C+d*D)/(p2)
    Rpq2 = np.dot(P-Q,P-Q)
    t1 = (p1*p2)*Rpq2/(p1+p2)
    Rab2 = np.dot(A-B, A-B)
    Rcd2 = np.dot(C-D, C-D)
    return N(a)*N(b)*N(c)*N(d)*2*(pi)**2.5*exp(-u(a,b)*Rab2 - u(c,d)*Rcd2)*F0(t1)/(p1*p2*sqrt(p1+p2))
class CGF:
    def __init__(self, center, exps, coefs):
        self.center = np.array(center, dtype=float)
        self.exps = np.array(exps, dtype=float)
        self.coefs = np.array(coefs, dtype=float)
        self.Nc = 1.0

    def normalize(self):
        s = 0.0
        A = self.center
        for i, ai in enumerate(self.exps):
            for j, aj in enumerate(self.exps):
                s += self.coefs[i] * self.coefs[j] * S(ai, aj, A, A)
        self.Nc = 1.0 / sqrt(s)
        return self.Nc
sto3g_1s = {
    "H": (
        [3.42525091, 0.62391373, 0.16885540],
        [0.15432897, 0.53532814, 0.44463454],
    ),
    "He": (
        [6.36242139, 1.15892300, 0.31364979],
        [0.15432897, 0.53532814, 0.44463454],
    ),
}
molecules = {
    "H2": {
        "ZA": 1, "ZB": 1, "charge": 0,
        "basisA": sto3g_1s["H"],
        "basisB": sto3g_1s["H"],
    },
    "HeH+": {
        "ZA": 2, "ZB": 1, "charge": +1,
        "basisA": sto3g_1s["He"],
        "basisB": sto3g_1s["H"],
    },
}
def setup_diatomic(R, ZA, ZB, basisA, basisB, charge=0):
    RA = np.zeros(3)
    RB = np.array([0.0, 0.0, float(R)])
    nuclei = [(RA, float(ZA)), (RB, float(ZB))]
    expsA, coefsA = basisA
    expsB, coefsB = basisB
    chiA = CGF(RA, expsA, coefsA); chiA.normalize()
    chiB = CGF(RB, expsB, coefsB); chiB.normalize()
    basis = [chiA, chiB]
    Ne = int(round(ZA + ZB - charge))
    return nuclei, basis, Ne
def Suv(c1, c2):
    val = 0.0
    for i, ai in enumerate(c1.exps):
        for j, bj in enumerate(c2.exps):
            val += c1.coefs[i] * c2.coefs[j] * S(ai, bj, c1.center, c2.center)
    return c1.Nc * c2.Nc * val
def Tuv(c1, c2):
    val = 0.0
    for i, ai in enumerate(c1.exps):
        for j, bj in enumerate(c2.exps):
            val += c1.coefs[i] * c2.coefs[j] * T(ai, bj, c1.center, c2.center)
    return c1.Nc * c2.Nc * val
def Vuv(c1, c2, nuclei):
    total = 0.0
    for (C, Z) in nuclei:
        C = np.array(C, dtype=float)
        val = 0.0
        for i, ai in enumerate(c1.exps):
            for j, bj in enumerate(c2.exps):
                val += c1.coefs[i] * c2.coefs[j] * V(ai, bj, c1.center, c2.center, C, Z)
        total += c1.Nc * c2.Nc * val
    return total
def build_one_electron(basis, nuclei):
    n = len(basis)
    S = np.zeros((n, n))
    T = np.zeros((n, n))
    V = np.zeros((n, n))
    for mu in range(n):
        for nu in range(n):
            S[mu, nu] = Suv(basis[mu], basis[nu])
            T[mu, nu] = Tuv(basis[mu], basis[nu])
            V[mu, nu] = Vuv(basis[mu], basis[nu], nuclei)
    H = T + V
    return S, T, V, H
def eri_abcd(c1, c2, c3, c4):
    val = 0.0
    for i, ai in enumerate(c1.exps):
        for j, bj in enumerate(c2.exps):
            for k, ck in enumerate(c3.exps):
                for l, dl in enumerate(c4.exps):
                    val += (
                        c1.coefs[i] * c2.coefs[j] * c3.coefs[k] * c4.coefs[l]
                        * two_electron_ERI(ai, bj, ck, dl, c1.center, c2.center, c3.center, c4.center)
                    )
    return c1.Nc * c2.Nc * c3.Nc * c4.Nc * val
def build_eri(basis):
    n = len(basis)
    ERI = np.zeros((n, n, n, n))
    for mu in range(n):
        for nu in range(n):
            for la in range(n):
                for si in range(n):
                    ERI[mu, nu, la, si] = eri_abcd(basis[mu], basis[nu], basis[la], basis[si])
    return ERI
def orth_X(S):
    eigval, U = np.linalg.eigh(S)
    eigval = np.maximum(eigval, 1e-14)
    return U @ np.diag(1.0 / np.sqrt(eigval)) @ U.T
def solve_roothaan(F, Xmat):
    Fp = Xmat.T @ F @ Xmat
    eps, Cp = np.linalg.eigh(Fp)
    C = Xmat @ Cp
    return eps, C
def build_density(C, Ne):
    if Ne % 2 != 0:
        raise ValueError
    nocc = Ne // 2
    Cocc = C[:, :nocc]
    return 2.0 * (Cocc @ Cocc.T)
def build_JK(ERI, Pmat):
    n = Pmat.shape[0]
    J = np.zeros((n, n))
    K = np.zeros((n, n))
    for mu in range(n):
        for nu in range(n):
            sJ = 0.0
            sK = 0.0
            for la in range(n):
                for si in range(n):
                    sJ += Pmat[la, si] * ERI[mu, nu, la, si]
                    sK += Pmat[la, si] * ERI[mu, la, nu, si]
            J[mu, nu] = sJ
            K[mu, nu] = sK
    return J, K
def build_fock(H, ERI, Pmat):
    J, K = build_JK(ERI, Pmat)
    Fmat = H + J - 0.5 * K
    return Fmat, J, K
def electronic_energy(Pmat, H, Fmat):
    return 0.5 * np.sum(Pmat * (H + Fmat))
def nuclear_repulsion(nuclei):
    Enuc = 0.0
    for i in range(len(nuclei)):
        Ri, Zi = nuclei[i]
        for j in range(i + 1, len(nuclei)):
            Rj, Zj = nuclei[j]
            Rij = np.linalg.norm(np.array(Ri) - np.array(Rj))
            Enuc += Zi * Zj / Rij
    return Enuc
def scf_rhf(S, H, ERI, nuclei, Ne, max_iter=50, conv=1e-10, damping=0.0):
    Xmat = orth_X(S)
    eps, C = solve_roothaan(H, Xmat)
    Pmat = build_density(C, Ne)
    Enuc = nuclear_repulsion(nuclei)
    E_old = 0.0
    for it in range(1, max_iter + 1):
        Fmat, J, K = build_fock(H, ERI, Pmat)
        eps, C = solve_roothaan(Fmat, Xmat)
        Pnew = build_density(C, Ne)
        if damping > 0.0:
            Pnew = (1 - damping) * Pnew + damping * Pmat
        E_elec = electronic_energy(Pnew, H, Fmat)
        E_tot = E_elec + Enuc
        dE = E_tot - E_old
        dP = np.max(np.abs(Pnew - Pmat))
        print(f"iter {it:2d}  E_tot = {E_tot:.12f}  dE = {dE:+.3e}  dP = {dP:.3e}")
        if abs(dE) < conv and dP < conv:
            return E_tot, E_elec, Enuc, eps, C, Pnew
        E_old = E_tot
        Pmat = Pnew
    raise RuntimeError
def run(name, R, *, ZA=None, ZB=None, charge=None, basisA=None, basisB=None, damping=0.0):
    spec = molecules[name].copy()
    if ZA is not None: spec["ZA"] = ZA
    if ZB is not None: spec["ZB"] = ZB
    if charge is not None: spec["charge"] = charge
    if basisA is not None: spec["basisA"] = basisA
    if basisB is not None: spec["basisB"] = basisB
    nuclei, basis, Ne = setup_diatomic(
        R, spec["ZA"], spec["ZB"], spec["basisA"], spec["basisB"], charge=spec["charge"]
    )
    S, T, V, H = build_one_electron(basis, nuclei)
    ERI = build_eri(basis)
    E_tot, E_elec, Enuc, eps, C, Pnew = scf_rhf(S, H, ERI, nuclei, Ne, damping=damping)
    print(f"\n{name} (R={R} bohr)")
    print(f"E_elec = {E_elec:.12f}")
    print(f"E_nuc  = {Enuc:.12f}")
    print(f"E_tot  = {E_tot:.12f}")
    return E_tot, E_elec, Enuc
if __name__ == "__main__":
    run("HeH+", 1.6, charge=+1, damping=0.0)
    print()
    run("H2", 1.4, damping=0.0)

iter  1  E_tot = -2.807818487891  dE = -2.808e+00  dP = 2.837e-01
iter  2  E_tot = -2.843187494189  dE = -3.537e-02  dP = 5.931e-02
iter  3  E_tot = -2.850054159588  dE = -6.867e-03  dP = 1.009e-02
iter  4  E_tot = -2.851204648334  dE = -1.150e-03  dP = 1.647e-03
iter  5  E_tot = -2.851392032492  dE = -1.874e-04  dP = 2.671e-04
iter  6  E_tot = -2.851422407865  dE = -3.038e-05  dP = 4.327e-05
iter  7  E_tot = -2.851427327970  dE = -4.920e-06  dP = 7.008e-06
iter  8  E_tot = -2.851428124813  dE = -7.968e-07  dP = 1.135e-06
iter  9  E_tot = -2.851428253865  dE = -1.291e-07  dP = 1.838e-07
iter 10  E_tot = -2.851428274765  dE = -2.090e-08  dP = 2.977e-08
iter 11  E_tot = -2.851428278150  dE = -3.385e-09  dP = 4.821e-09
iter 12  E_tot = -2.851428278698  dE = -5.482e-10  dP = 7.808e-10
iter 13  E_tot = -2.851428278787  dE = -8.878e-11  dP = 1.265e-10
iter 14  E_tot = -2.851428278801  dE = -1.438e-11  dP = 2.048e-11

HeH+ (R=1.6 bohr)
E_elec = -4.101428278801
E_nuc  = 1.250000000000
E_tot  =